# 3D Earth model plotting and analysis

Avni has tools to visualize and interact with global models that are written in our custom hdf5 format (Citation of Tools paper?). Users have two options to use the tools. First, if a user has the model file saved locally, they can use Avni's Model3D class which includes a variety of methods including reading, writing, and plotting. Typically, 3D global models have large files so it may not always be desirable to download them locally. In this case, users can use Avni's API to interact with our database of global Earth models. Below, we will show examples of both use cases. 

## 1) Plotting with local models

### Import avni modules

In [1]:
import avni
from avni import models
from avni.models import Model3D
from avni.plots import plot1section,plot1globalmap

### Read tomography model S362ANI+M (Moulik and Ekstrom, 2016)

In [2]:
#m = model3d('/Users/rmaguire/Desktop/Manuscripts/Avni/S362ANI+M.BOX25km_PIX1X1.rem3d.nc4')
#m = models.model3d('/Users/rmaguire/Desktop/Manuscripts/Avni/S362ANI+M.BOX25km_PIX1X1.rem3d.nc4')
m = Model3D('/Users/rmaguire/Desktop/Manuscripts/Avni/S362ANI+M.BOX25km_PIX1X1.rem3d.nc4')

Traceback (most recent call last):
  File "/Users/rmaguire/Tools/avni/avni/models/model3d.py", line 132, in read
    self.readhdf5(hf)
  File "/Users/rmaguire/Tools/avni/avni/models/model3d.py", line 212, in readhdf5
    raise ValueError("... choose one from multiple query "+strsum)
ValueError: ... choose one from multiple query  , 410 , 650 , CMB , as , depth , latitude , longitude , pixel_width , vs , vsh , vsv

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/rmaguire/Tools/avni/avni/models/realization.py", line 109, in read
    self.decode_units()
  File "/Users/rmaguire/Tools/avni/avni/models/realization.py", line 133, in decode_units
    raise ValueError('decoding units not possible without '+key+' initialized from model file and attributes.ini')
ValueError: decoding units not possible without absolute_unit initialized from model file and attributes.ini



In [3]:
#latitude,longitude,azimuth,gcdelta,model,parameter,
#plot1section(latitude=0,longitude=0,azimuth=0,gcdelta=90,model='S362ANI+M.BOX25km_PIX1X1.rem3d.nc4',parameter='vs')

In [4]:
directory = avni.tools.get_filedir()
filename = 'S362ANI+M.BOX25km_PIX1X1.rem3d.nc4'
avni.data.update_file(filename)

200

In [5]:
file = directory+'/'+filename
S362ANIM = avni.models.Model3D(file)

Traceback (most recent call last):
  File "/Users/rmaguire/Tools/avni/avni/models/model3d.py", line 132, in read
    self.readhdf5(hf)
  File "/Users/rmaguire/Tools/avni/avni/models/model3d.py", line 212, in readhdf5
    raise ValueError("... choose one from multiple query "+strsum)
ValueError: ... choose one from multiple query  , as , depth , latitude , longitude , topo410 , topo650 , topoCMB , vs , vsh , vsv

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/rmaguire/Tools/avni/avni/models/realization.py", line 88, in read
    self.readascii(file)
  File "/Users/rmaguire/Tools/avni/avni/models/realization.py", line 268, in readascii
    model=read3dmodelfile(modelfile)
  File "/Users/rmaguire/Tools/avni/avni/models/common.py", line 117, in read3dmodelfile
    with open(modelfile) as f: lines = f.readlines()
  File "/Users/rmaguire/anaconda2/envs/avni/lib/python3.7/codecs.py", line 322, in decode
    (result, consum

## 2) Plotting with API

### Import avni modules

In [6]:
from avni.api.client import Client as avapi
from avni.api.model import Model

### Connect to the avni server to access model database

Connecting the avni server, which hosts a database of 3D global tomography models, requires an "API key". To acquire your API key, register at https://maurya.umd.edu/tools/applets/. Here we will use an activated API key as an example.

In [7]:
conn=avapi(api_key='ae691a174e5fef17f28bfad58f66956b5ca820f063fbb2e6a41dfffc17c11f6d')

avni api is live at 
http://maurya.umd.edu:41559
API Key validated


### Create an instance of the Model class, and plot a cross section
An instance of the avni api Model class is initialized by providing the api connection as an argument. A variety of useful class methods then become available. For example, we can list all of the available models on the avni server using the $\mathrm{listModels()}$ method, which returns a dictionary object containing the names and metadata of available models.

To create a cross section of one of the models, we will call the $\mathrm{crossSection()}$ method. This method requires an argument dictionary that specifies the model name and the geometry of the cross section. The required arguments are the latitude and longitude of the center of the cross section, the azimuth, and the great circle distance. 

In [8]:
ModelInstance = Model(conn)
models_dict = ModelInstance.listModels() #list all available models
print(models_dict)

#create cross section
args = {}                      #empty argument dictionary
args['model'] = 'S362ANI+M'    #tomography model
args['lat'] = 42.2             #latitude of center point
args['lon'] = 232.0            #longitude of center point
args['azimuth'] = 80.0         #azimuth of slice
args['gcdelta'] = 100.0        #great circle distance of the slice in degrees

cross_sec = ModelInstance.crossSection(args)

{'3d': {'available': ['S40RTS', 'GAP_P4', 'S362ANI+M', 'MITP08'], 'details': {'S40RTS': {'parameters': ['vp', 'vs', 'rho'], 'vp': {'refmodel': 'PREM', 'description': 'Vp anomaly scaled from Vs', 'absolute_unit': 'm/s', 'unit': 'percent'}, 'vs': {'refmodel': 'PREM', 'description': 'Voigt-averaged Shear-Wave Velocity anomaly', 'absolute_unit': 'm/s', 'unit': 'percent'}, 'rho': {'refmodel': 'PREM', 'description': 'density anomaly scaled from Vs', 'absolute_unit': 'kg/m^3', 'unit': 'percent'}, 'kernel': 'BOX25km_PIX1X1'}, 'GAP_P4': {'parameters': ['vp'], 'vp': {'refmodel': 'None', 'description': 'Voigt-averaged Compressional-Wave Velocity', 'absolute_unit': 'm/s', 'unit': 'percent'}, 'kernel': 'BOX25km_PIX0.625X0.625'}, 'S362ANI+M': {'parameters': ['vs', 'vsh', 'vsv'], 'vs': {'refmodel': 'STW105', 'description': 'Voigt-averaged Shear-Wave Velocity', 'absolute_unit': 'm/s', 'unit': 'percent'}, 'vsh': {'refmodel': 'STW105', 'description': 'Horizontally polarized Shear-Wave Velocity', 'absolu

In [9]:
cross_sec.keys()

dict_keys(['vs', 'parameter', 'lat', 'lon', 'depth', 'call_complete', 'theta'])